[![img/pythonista.png](img/pythonista.png)](https://www.pythonista.io)

# *Pydantic* y *SQLAlchemy*.

Es posible relacionar un esquema definido con *Pydantic* a un modelo creado con *SQLAlchemy*.

## El atributo   ```Config``` de ```pydantic.BaseModel```.

Este atributo se define como una clase y contiene metadatos que sirven para configurar a las subclases de ```pydantic.BaseModel```. 

Uno de atributos importantes de ```Config``` es ```orm_mode```, el cual, cuando su valor es ```Tue```, indica que los objetos instanciados pueden interactuar con un ORM.

## El atributo ```from_orm()```.

Este atributo permite crear objetos instanciados de subclases de ```BaseModel``` ligados a un modelo de *ORM*.

## Ejemplo ilustrativo.

In [1]:
import sqlalchemy as sa
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from pydantic import BaseModel, Field, PositiveInt, validator
from typing import Literal

### Creación del modelo ```Alumno``` con *SQLAlchemy*.

In [2]:
Base = declarative_base()

In [3]:
class Alumno(Base):
    __tablename__ = 'alumnos'
    cuenta = sa.Column(sa.Integer, primary_key=True)
    nombre = sa.Column(sa.String(50))
    primer_apellido = sa.Column(sa.String(50))
    segundo_apellido = sa.Column(sa.String(50))
    carrera = sa.Column(sa.String(50))
    semestre = sa.Column(sa.Integer)
    promedio = sa.Column(sa.Float)
    al_corriente = sa.Column(sa.Boolean)

### Inicio de la sesión.

In [4]:
engine = sa.create_engine('sqlite:///db.sqlite3')

In [5]:
Session = sessionmaker(bind=engine)

In [6]:
session = Session()

In [7]:
Base.metadata.create_all(engine)

### Creación del esquema ```AlumnoSchema``` con *Pydantic*.

In [8]:
CARRERAS = ['Arquitectura',
            'Diseño',
            'Sistemas',
            'Derecho',
            'Actuaría']

In [9]:
class AlumnoSchema(BaseModel):
    cuenta: int
    nombre: str
    primer_apellido: str
    segundo_apellido: str = ''
    carrera: Literal[tuple(CARRERAS)]
    semestre: PositiveInt
    promedio: float = Field(ge=0, le=10)
    al_corriente: bool
        
    class Config:
        orm_mode = True

* La siguiente celda creará un objeto ```list``` con el nombre ```alumnos``` que contendrá las instancias de ```Alumno``` construidas a partir de todos los registros de la tabla ```alumnos``` en la base de datos.

### Extracción del estado del modelo desde la base de datos.

In [10]:
alumnos = session.query(Alumno).filter(Alumno.cuenta).all()

In [11]:
alumnos

### Serialización del estado del modelo.

In [12]:
datos_alumnos = [AlumnoSchema.from_orm(alumno) for alumno in alumnos]

In [13]:
datos_alumnos

[AlumnoSchema(cuenta=1231221, nombre='Pedro', primer_apellido='Solis', segundo_apellido='Cabañas', carrera='Arquitectura', semestre=3, promedio=7.8, al_corriente=False),
 AlumnoSchema(cuenta=1231222, nombre='Yolanda', primer_apellido='Jiménez', segundo_apellido='Lerdo', carrera='Actuaría', semestre=3, promedio=6.0, al_corriente=False),
 AlumnoSchema(cuenta=1231223, nombre='Juan', primer_apellido='Ramos', segundo_apellido='Breña', carrera='Sistemas', semestre=9, promedio=8.6, al_corriente=True),
 AlumnoSchema(cuenta=1231224, nombre='Mayra Jimena', primer_apellido='Cervantes', segundo_apellido='Lisama', carrera='Derecho', semestre=12, promedio=9.2, al_corriente=True)]

### Validación y adición de un nuevo registro a la base de datos.

In [14]:
datos = {'cuenta': 1234567,
         'nombre': 'Juan',
         'primer_apellido': 'Pérez',
         'segundo_apellido': 'López',
         'carrera': 'Sistemas',
         'semestre': 7,
         'promedio': 6.5,
        'al_corriente': True}

In [15]:
session.add(Alumno(**AlumnoSchema(**datos).dict()))

In [16]:
session.commit()

In [17]:
%load_ext sql

In [18]:
%sql sqlite:///db.sqlite3

In [19]:
%sql select * from alumnos

 * sqlite:///db.sqlite3
Done.


cuenta,nombre,primer_apellido,segundo_apellido,carrera,semestre,promedio,al_corriente
1231221,Pedro,Solis,Cabañas,Arquitectura,3,7.8,0
1231222,Yolanda,Jiménez,Lerdo,Actuaría,3,6.0,0
1231223,Juan,Ramos,Breña,Sistemas,9,8.6,1
1231224,Mayra Jimena,Cervantes,Lisama,Derecho,12,9.2,1
1234567,Juan,Pérez,López,Sistemas,7,6.5,1


### Eliminación de un registro de la base de datos.

In [20]:
alumno_borrar = session.query(Alumno).filter_by(cuenta=1234567).first()
session.delete(alumno_borrar)
session.commit()

In [21]:
%sql select * from alumnos

 * sqlite:///db.sqlite3
Done.


cuenta,nombre,primer_apellido,segundo_apellido,carrera,semestre,promedio,al_corriente
1231221,Pedro,Solis,Cabañas,Arquitectura,3,7.8,0
1231222,Yolanda,Jiménez,Lerdo,Actuaría,3,6.0,0
1231223,Juan,Ramos,Breña,Sistemas,9,8.6,1
1231224,Mayra Jimena,Cervantes,Lisama,Derecho,12,9.2,1


In [22]:
session.close()

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2022.</p>